In [1]:
# %load main.py
from model import *
from data import *

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [ ]:
# 데이터 부풀리기
# 1) rotation_range : 지정된 각도 범위 내에서 임의로 원본 이미지를 회전함.
# 2) width_shift_range : 지정된 수평방향 이동 범위 내에서 임의로 원본 이미지를 이동시킴. 수치는 전체 넓이의 비율로 나타냄
# 3) height_shift_range : 지정된 수직방향 이동 범위 내에서 임의로 원본 이미지를 이동시킴. 수치는 전체 높이의 비율로 나타냄.
# 4) shear_range : 밀림 강도 범위 내에서 임의로 원본 이미지를 변형시킴. 시계 반대 방향으로 밀림 강도를 라디안으로 나타냄.
# 5) zoom_range : 지정된 확대/축소 범위 내에서 임의로 원본 이미지를 확대/축소함. '1-수치'부터 '1+수치'사이 범위로 확대/축소함.
# 6) horizontal_flip : 수평방향으로 뒤집기를 함.
# 7) fill_mode : 이미지를 회전, 이동하거나 축소할 때 생기는 공간을 채우는 방식
# fill_mode='nearest' : aaaaaaaa|abcd|dddddddd
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

# Generator : 데이터셋 생성
# trainGenerator : 트레이닝 데이터셋 생성
# 2 : batch size
# save_to_dir : 저장 경로 지정
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)

model = unet()

# ModelCheckpoint :  매 epoch마다 모델의 가중치를 저장하도록함.
# ModelCheckpoint(filepath, monitor, verbose, save_best_only)
# 1) filepath : 인자에서 실제 가중치를 저장할 경로를 지정함.
# 2) monitor : 관찰하고자 하는 항목
# 3) verbose : 1일 시 학습을 진행하는동안 가중치 파일이 업데이트 될 때마다 텍스트를 출력함.
# 4) save_best_only : True시 validation set에서 가장 정확도가 높았던 시행에서의 가중치만을 저장함.
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)

# Generator로 생성된 배치로 학습시킬 경우 fit_generator() 함수를 사용함.
# model.fit_generator(첫번째인자, steps_per_epoch, epochs, callbacks)
# 1) 첫 번째 인자 : 훈련데이터셋을 제공할 generator 지정.
# 2) steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. (훈련 샘플 수/배치 사이즈)
# 3) epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정.
# 4) callbacks : 콜백할 위치 지정.
model.fit_generator(myGene,steps_per_epoch=300,epochs=1,callbacks=[model_checkpoint])

# test할 데이터셋 경로
testGene = testGenerator("data/membrane/test")

# predict_generator : 데이터 생성기로부터 입력 샘플에 대한 예측을 생성함.
# predict_generator(self, max_queue_size, verbose)
# 1) max_queue_size : 생성기 큐의 최대 크기
# 2) verbose : 1일 시 상세 정보 모드를 업데이트 될 때마다 텍스트로 출력함.
results = model.predict_generator(testGene,30,verbose=1)

# 결과값 저장
saveResult("data/membrane/test",results)